In [17]:
import requests
import re
import pandas as pd
import time
from bs4 import BeautifulSoup

# Get all the symbols

In [18]:
response = requests.get("https://coinmarketcap.com/currencies/")
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
main_table = [t for t in soup.find_all('table') if t.has_attr('id') and t['id'] == 'currencies']
assert len(main_table) == 1, "Only one main table"
main_table = main_table[0]

In [20]:
coins = [t for t in main_table.find_all('tr') if t.has_attr('id')]
coin_aliases = [re.search("\/currencies\/(.*)\/", t.find_all('a')[0]['href']).group(1) for t in coins]
coin_aliases

['bitcoin',
 'ethereum',
 'ripple',
 'litecoin',
 'ethereum-classic',
 'dash',
 'nem',
 'monero',
 'iota',
 'stratis',
 'bitconnect',
 'zcash',
 'bitshares',
 'antshares',
 'bytecoin-bcn',
 'steem',
 'qtum',
 'waves',
 'stellar',
 'siacoin',
 'dogecoin',
 'lisk',
 'byteball',
 'factom',
 'decred',
 'komodo',
 'digibyte',
 'gamecredits',
 'pivx',
 'nxt',
 'bancor',
 'lykke',
 'chaincoin',
 'bitcoindark',
 'leocoin',
 'decent',
 'peercoin',
 'syscoin',
 'emercoin',
 'nexus',
 'ark',
 'ubiq',
 'verge',
 'reddcoin',
 'namecoin',
 'monacoin',
 'gulden',
 'asch',
 'peerplays-ppy',
 'cloakcoin',
 'e-dinar-coin',
 'daxxcoin',
 'sibcoin',
 'counterparty',
 'skycoin',
 'blocknet',
 'library-credit',
 'bitbay',
 'omni',
 'ybcoin',
 'potcoin',
 'zcoin',
 'mooncoin',
 'blackcoin',
 'viacoin',
 'burst',
 'crown',
 'faircoin',
 'dubaicoin-dbix',
 'vertcoin',
 'groestlcoin',
 'iocoin',
 'firstcoin',
 'golos',
 'expanse',
 'elastic',
 'nav-coin',
 'novacoin',
 'salus',
 'digitalnote',
 'gridcoin',
 'e-

# Get all the market prices

In [21]:
# Initialize variables
LEVEL=5
NUM_OF_COINS=50

In [22]:
def get_pair_info(tr_text):
    """
    Get pair info from html text of <tr>
    """
    tds = tr_text.find_all('td')
    exchange = tds[1].text
    pair = tds[2].text
    volume = float(tds[3].find('span')['data-usd']) if tds[3].find('span')['data-usd'] != '?' else 0
    price = float(tds[4].find('span')['data-usd']) if tds[4].find('span')['data-usd'] != '?' else 0
    share = float(tds[5].text[0:-1])
    time = tds[6].text

    return exchange, pair, volume, price, share, time

In [23]:
def get_records(coin_alias):
    """
    Get the highest and lowest 5 prices among all exchanges.
    """
    # Get the market html
    coin_market_ret = requests.get("https://coinmarketcap.com/currencies/%s/#markets" % coin_alias)
    assert coin_market_ret.status_code == 200, "Status code (%d) is not 200.\n%s" % (coin_market_ret.status_code, coin_market_ret.text)
    coin_market_soup = BeautifulSoup(coin_market_ret.text, 'html.parser')
    coin_table = [t for t in coin_market_soup.find_all('table') if t.has_attr('id') and t['id'] == 'markets-table']
    assert len(coin_table) == 1, "Only one main table"
    coin_table = coin_table[0]
    # Get all the records
    coin_records = coin_table.find_all('tr')
    records = pd.DataFrame(columns=['exchange', 'pair', 'volume', 'price', 'share', 'last_updated'],
                           data=[get_pair_info(t) for t in coin_records 
                                 if len(t.find_all('td')) > 0])
    # Take only recent records
    records = records[(records['last_updated'] == "Recently") & (records['price'] > 0)]
    return records

In [24]:
coin_records = {}
for coin_alias in coin_aliases[0:NUM_OF_COINS]:
    coin_records[coin_alias] = get_records(coin_alias)
    time.sleep(5)

In [64]:
arb_pairs = pd.DataFrame(columns=(["h%d" % (t+1) for t in range(0,LEVEL)]
                                  +["l%d" % (t+1) for t in range(0,LEVEL)]
                                  +["pd%d" % (t+1) for t in range(0,LEVEL)]))
for coin_alias in coin_aliases[0:NUM_OF_COINS]:
    records = coin_records[coin_alias]
    highest = records.sort_values('price', ascending=False).head(LEVEL)
    lowest = records.sort_values('price').head(LEVEL)
    highest_desc = highest.apply(lambda x: "%s (%.6f)" % (x["exchange"], x["price"]), axis=1)
    lowest_desc = lowest.apply(lambda x: "%s (%.6f)" % (x["exchange"], x["price"]), axis=1)
    pd = records.sort_values('price', ascending=False).head(LEVEL)['price'].values
    pd /= records.sort_values('price').head(LEVEL)['price'].values
    pd = (pd - 1) * 100
    highest_desc = (highest_desc.tolist() + ['???'] * LEVEL)[0:LEVEL]
    lowest_desc = (lowest_desc.tolist() + ['???'] * LEVEL)[0:LEVEL]
    pd = (pd.tolist() + [0] * LEVEL)[0:LEVEL]
    arb_pairs.loc[coin_alias] = highest_desc + lowest_desc + pd
    time.sleep(5)
arb_pairs

,h1,h2,h3,h4,h5,l1,l2,l3,l4,l5,pd1,pd2,pd3,pd4,pd5
bitcoin,Poloniex (3990.580000),xBTCe (2780.270000),C-CEX (2626.400000),Livecoin (2573.080000),Bittylicious (2553.610000),CoinGather (1410.540000),Bitcoin Growth Fund Exchange (1570.870000),Cryptopia (1576.360000),Poloniex (1693.850000),Livecoin (1856.790000),1.829115e+02,7.698918e+01,6.661169e+01,5.190719e+01,37.528207
ethereum,Decentrex (7608.260000),Novaexchange (628.305000),EtherDelta (604.415000),EtherDelta (399.989000),CryptoDerivatives (376.716000),CryptoDerivatives (0.121732),CryptoDerivatives (3.737180),Novaexchange (4.258050),Quoine (9.774240),Quoine (10.749300),6.249908e+06,1.671228e+04,1.409464e+04,3.992277e+03,3404.563088
ripple,BTCXIndia (0.188233),Cryptomate (0.182348),BTER (0.179491),Bitsane (0.179195),Bitsane (0.177754),Rippex (0.151575),Mr. Ripple (0.153744),Mr. Ripple (0.155813),Mr. Ripple (0.156410),Gatehub (0.157551),2.418473e+01,1.860495e+01,1.519642e+01,1.456748e+01,12.823149
litecoin,Cryptopia (1894970.000000),CoinGather (842209.000000),Novaexchange (89498.600000),Cryptopia (81510.600000),Cryptopia (1504.410000),Cryptopia (4.047430),DCExchange (4.250000),Cryptopia (6.875950),Cryptopia (7.068260),Novaexchange (7.238380),4.681899e+07,1.981658e+07,1.301518e+06,1.153092e+06,20683.794164
ethereum-classic,C-CEX (18.950000),USD X (16.992900),BTC Markets (16.362600),BTC Markets (16.285100),Cryptomate (16.249600),CoinExchange (12.029400),HitBTC (13.687800),Gatehub (13.779800),Cryptopia (14.162400),Bittrex (14.275900),5.753072e+01,2.414632e+01,1.874338e+01,1.498828e+01,13.825398
dash,Novaexchange (1278.790000),CoinExchange (217.951000),Bittylicious (209.673000),OpenLedger DEX (175.927000),USD X (175.853000),OpenLedger DEX (0.122324),OpenLedger DEX (4.850580),Crypto Dao (16.619400),COSS (58.573900),Livecoin (59.264900),1.045312e+06,4.393298e+03,1.161616e+03,2.003505e+02,196.723693
nem,Cryptopia (0.182777),LiteBit.eu (0.115635),Zaif (0.115013),Cryptopia (0.111701),BTER (0.111125),BTER (0.101646),COSS (0.107462),HitBTC (0.107727),COSS (0.107735),Poloniex (0.107913),7.981721e+01,7.605479e+00,6.763393e+00,3.681255e+00,2.976472
monero,SouthXchange (35.000000),Cryptomate (34.803100),Bitsquare (32.973700),Tux Exchange (32.513800),BTER (31.853300),Poloniex (27.174400),Cryptopia (29.230000),Poloniex (30.245600),BTC Alpha (30.451000),Poloniex (30.741900),2.879769e+01,1.906637e+01,9.019824e+00,6.774162e+00,3.615261
iota,Bitfinex (0.163295),Bitfinex (0.162569),IOTA Exchange (0.162569),Bitfinex (0.161440),???,Bitfinex (0.161440),Bitfinex (0.162569),IOTA Exchange (0.162569),Bitfinex (0.163295),???,1.149034e+00,0.000000e+00,0.000000e+00,-1.135981e+00,0.000000
stratis,Cryptopia (4.763050),SouthXchange (3.603900),Bittylicious (3.368150),POSWallet (3.360280),Cryptopia (3.271200),Cryptopia (2.007850),Livecoin (2.894500),SouthXchange (2.945630),Livecoin (2.995550),Livecoin (3.009800),1.372214e+02,2.450855e+01,1.434396e+01,1.217573e+01,8.684962


In [66]:
arb_pairs[['h3', 'l3', 'pd3']].round(2)

,h3,l3,pd3
bitcoin,C-CEX (2626.400000),Cryptopia (1576.360000),66.61
ethereum,EtherDelta (604.415000),Novaexchange (4.258050),14094.64
ripple,BTER (0.179491),Mr. Ripple (0.155813),15.20
litecoin,Novaexchange (89498.600000),Cryptopia (6.875950),1301517.96
ethereum-classic,BTC Markets (16.362600),Gatehub (13.779800),18.74
dash,Bittylicious (209.673000),Crypto Dao (16.619400),1161.62
nem,Zaif (0.115013),HitBTC (0.107727),6.76
monero,Bitsquare (32.973700),Poloniex (30.245600),9.02
iota,IOTA Exchange (0.162569),IOTA Exchange (0.162569),0.00
stratis,Bittylicious (3.368150),SouthXchange (2.945630),14.34
